In [ ]:
# Basic data handling
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from matplotlib.ticker import MaxNLocator

# Utilities
import json
import re
import math
from collections import Counter
from concurrent.futures import ThreadPoolExecutor, as_completed

# Parallel processing and serialization
import tqdm
import joblib

# Statistical modeling
from scipy.stats.qmc import LatinHypercube


In [ ]:
with open('官方媒体/官媒清单.txt', 'r') as f:
    data = f.read()
    data = data.split('\n')
m_media = [item for item in data if item != '']
data = pd.read_excel('./output/data.xlsx')

In [ ]:
mid_name = {}
for mid, name in data[['mid', 'user_name']].values:
    mid_name[mid] = name
mid_verify_typ = {}
for mid, verified in data[['mid', 'verify_typ']].values:
    mid_verify_typ[mid] = verified

In [ ]:
forwarding = pd.read_csv('./data/新冠topic_reposts_用户数据/新冠topic_reposts.csv')
forwarding['origin_name'] = forwarding['origin_mid'].apply(lambda x: mid_name.get(x, ''))
forwarding['origin_verify_typ'] = forwarding['origin_mid'].apply(lambda x: mid_verify_typ.get(x, ''))
forwarding = forwarding[forwarding['认证类型'] != '暂不爬取']
# 去除认证类型为缺失值的行
forwarding = forwarding.dropna(subset=['认证类型'])
forwarding['认证类型'] = forwarding['认证类型'].astype('int')

In [ ]:
# 1和3还有在官媒清单里代表官媒，[0, 200, 220]代表自媒体，-1代表普通用户
forwarding['d_type'] = 'None'
forwarding.loc[forwarding['认证类型'] == 1, 'd_type'] = 'm_media'
forwarding.loc[forwarding['认证类型'] == 3, 'd_type'] = 'w_media'
forwarding.loc[forwarding['认证类型'] == 0, 'd_type'] = 'w_media'
forwarding.loc[forwarding['认证类型'] == 200, 'd_type'] = 'w_media'
forwarding.loc[forwarding['认证类型'] == 220, 'd_type'] = 'w_media'
forwarding.loc[forwarding['认证类型'] == -1, 'd_type'] = 'o_people'
forwarding.loc[forwarding['user_name'].isin(m_media), 'd_type'] = 'm_media'

In [ ]:
m_media_name = forwarding[forwarding['d_type'] == 'm_media']['user_name']
w_media_name = forwarding[forwarding['d_type'] == 'w_media']['user_name']
m_media_name = m_media_name.drop_duplicates() 
m_media_name = m_media_name.to_list()
m_media_name = m_media_name + m_media
m_media_name = list(set(m_media_name))

w_media_name = w_media_name.drop_duplicates()
w_media_name = w_media_name.to_list()
w_media_name = list(set(w_media_name))

In [ ]:
np.random.seed(1234)
forwarding['o_type'] = 'None'
forwarding.loc[forwarding['origin_verify_typ'] == '没有认证', 'o_type'] = 'o_people'
# forwarding.loc[forwarding['origin_verify_typ'] != '没有认证', 'o_type'] = 'w_media'
forwarding.loc[forwarding['origin_name'].isin(w_media_name), 'o_type'] = 'w_media'
forwarding.loc[forwarding['origin_name'].isin(m_media_name), 'o_type'] = 'm_media'
# 统计每天的转发量，利用publish_time获取日期
forwarding['publish_time'] = pd.to_datetime(forwarding['publish_time'])
forwarding['date'] = forwarding['publish_time'].dt.date
forwarding['date'] = pd.to_datetime(forwarding['date'])
forwarding_sample = forwarding[('2022-10-01' <= forwarding['date'])&(forwarding['date'] <= '2023-01-18')]
# forwarding_sample.groupby('date')['is_mainstream'].sum().sort_values(ascending=False).reset_index().to_csv('官方媒体/官媒转发量.csv', index=False)
forwarding_sample = forwarding_sample.sample(frac=0.3)

In [ ]:
forwarding_sample.loc[forwarding_sample['o_type'] == 'None', 'o_type'] = 'w_media'
links = forwarding_sample[['origin_name', 'o_type', 'user_name', 'd_type']]

In [ ]:
links = links[links['d_type'] != 'None']
links = links.drop_duplicates()
links = links[~ ((links['o_type'] == 'o_people') & (links['d_type'] == 'o_people'))]
links

In [ ]:
data_sentiment = pd.read_csv('./output/sentiment/data_sentiment.csv')
data_risk_mainstream = pd.read_csv('./output/risk_media/data_risk_mainstream.csv')
data_risk_wemedia = pd.read_csv('./output/risk_media/data_risk_wemedia.csv')
data_sentiment.dropna(subset=['sentiment'], inplace=True)
data_risk_mainstream.dropna(subset=['risk'], inplace=True)
data_risk_wemedia.dropna(subset=['risk'], inplace=True)

In [ ]:
# 如果用户重复出现，那么取最近值
user_sentiment = {}
for name, sentiment in data_sentiment[['user_name', 'sentiment']].values:
    if sentiment == 'high':
        sentiment = 'H'
    elif sentiment == 'low':
        sentiment = 'L'
    else:
        sentiment = 'M'
    user_sentiment[name] = sentiment
    
risk_mainstream = {}
for name, risk in data_risk_mainstream[['user_name', 'risk']].values:
    if risk == 'yes':
        risk = 'R'
    else:
        risk = 'NR'
    risk_mainstream[name] = risk
    
risk_wemedia = {}
for name, risk in data_risk_wemedia[['user_name', 'risk']].values:
    # 对risk进行重编码
    if risk == 'yes':
        risk = 'R'
    else:
        risk = 'NR'
    risk_wemedia[name] = risk

In [ ]:
# plot the repost number distribution

# Ensure that Times New Roman is used as the font for all plot text
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12  # You can adjust the base font size here if needed

# Assuming 'forwarding' DataFrame and 'date' column are already defined
# Create a 'year_month' column for grouping by year and month
forwarding['year_month'] = forwarding['date'].dt.to_period('M')

# Calculate the count of forwarding actions per month
monthly_count = forwarding.groupby('year_month').size()

# Start plotting
plt.figure(figsize=(12, 8))  # Set the figure size for better detail

# Plot data
plt.plot(monthly_count.index.to_timestamp(), monthly_count.values, color='royalblue', marker='o', linestyle='-')

# Improve the formatting of the x-axis to handle dates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # Adjust interval for less crowded x-axis
plt.xticks(rotation=45, ha='right')  # Rotate the x-axis labels for better readability

# Adding a title and labels with increased font sizes for readability
plt.title('Monthly Forwarding Count Distribution', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=14)
plt.ylabel('Forwarding Count', fontsize=14)

# Optionally, set the x-axis to only have a maximum number of date ticks
plt.gca().xaxis.set_major_locator(MaxNLocator(6))

# Remove grid lines for a cleaner look
plt.grid(False)

# Enhancing visual style by removing top and right borders
sns.despine()

# Save the figure with a high resolution of 300 DPI
plt.savefig('./graph/forwarding_count_distribution.png', format='png', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

## Model Initialization

In [ ]:
class Node:
    """
    A node in a network that represents an entity, which could be a media outlet or an individual,
    capable of influencing or being influenced in terms of risk and sentiment.
    """
    def __init__(self, name, node_type, risk=None, sentiment=None, sigma=None):
        """
        Initializes a new instance of the Node class.
        
        Args:
            name (str): The unique name or identifier for the node.
            node_type (str): The type of the node (e.g., 'm_media', 'w_media', 'o_people').
            risk (str, optional): Initial risk status of the node (e.g., 'R', 'NR'). Defaults to None.
            sentiment (str, optional): Initial sentiment of the node (e.g., 'H', 'M', 'L'). Defaults to None.
            sigma (float, optional): A threshold parameter used in decision-making processes. Defaults to None.
        """
        self.name = name
        self.type = node_type
        self.risk = risk
        self.sentiment = sentiment
        self.sigma = sigma
        self.influencers = []  # List to store nodes that influence this node.

    def add_influencer(self, influencer):
        """
        Adds a node to the list of influencers for this node.
        
        Args:
            influencer (Node): The node to be added as an influencer.
        """
        self.influencers.append(influencer)

    def calculate_n_major(self):
        """
        Calculates the highest number of similar sentiments among the influencers of this node.
        
        Returns:
            int: The count of the most frequent sentiment category among the influencers.
        """
        sentiment_counter = Counter(inf.sentiment for inf in self.influencers if inf.sentiment)
        return max(sentiment_counter.values()) if sentiment_counter else 0

    def calculate_influencers_sentiment_distribution(self):
        """
        Calculates the distribution of sentiments among the influencers.
        
        Returns:
            tuple: A tuple containing counts of high, low, and middle sentiments respectively.
        """
        sentiment_counts = Counter(inf.sentiment for inf in self.influencers if inf.sentiment)
        return sentiment_counts.get('H', 0), sentiment_counts.get('L', 0), sentiment_counts.get('M', 0)

    def update_m_media(self, params):
        """
        Updates the risk status of this node if it is of type 'm_media' based on external parameters.
        
        Args:
            params (dict): Parameters that include alpha and the differential sentiment count.
        """
        if self.type == 'm_media':
            alpha = params['alpha']
            d = params['d']  # Differential in sentiment counts
            if d > 0:
                Pu = 1 - math.exp(-alpha * d)
                if Pu > np.random.random():
                    self.risk = 'NR'

    def update_w_media(self, params):
        """
        Updates the risk status for 'w_media' type nodes based on the sentiment distribution among influencers.
        
        Args:
            params (dict): Parameters including the beta coefficient and normalized sentiment metrics.
        """
        if self.type == 'w_media':
            n_high_p = params['n_high'] / (params['n_high'] + params['n_low'] + params['n_middle'])
            Pv = 1 / (1 + np.exp(-params['beta'] * n_high_p))
            if Pv > np.random.uniform(0.5, 1):
                self.risk = 'R'
            else:
                self.risk = 'NR'

    def update_o_people(self, params):
        """
        Updates the sentiment of 'o_people' type nodes based on the influence of risk status from influencers.
        
        Args:
            params (dict): Contains parameters such as theta which influences the sentiment update process.
        """
        if self.type == 'o_people':
            r_1 = sum(1 if inf.risk == 'R' else -1 for inf in self.influencers) / (len(self.influencers) + 1)
            g_1 = 1 / (1 + np.exp(-params['theta'] * r_1)) if r_1 != 0 else 0
            if g_1 > np.random.uniform(0, 1):
                self.sentiment = np.random.choice(['L', 'H'])
            else:
                self.sentiment = 'M'
            
            sentiments = [inf.sentiment for inf in self.influencers if inf.sentiment]
            eta = sum(1 for inf in self.influencers if inf.sentiment == self.sentiment) / len(sentiments) if sentiments else 0
            if eta > self.sigma:
                sentiment_counter = Counter([inf.sentiment for inf in self.influencers if inf.sentiment])
                self.sentiment = max(sentiment_counter, key=sentiment_counter.get)
    
    def update_w_media_gv(self, params, government_effect_w):
        """
        Updates the risk status for 'w_media' type nodes considering government intervention effect alongside the basic sentiment distribution.
        
        Args:
            params (dict): Parameters including sentiment proportions and beta coefficient.
            government_effect_w (float): Adjustment to the influence probability due to government intervention.
        """
        if self.type == 'w_media':
            n_high_p = params['n_high'] / (params['n_high'] + params['n_low'] + params['n_middle'])
            Pv = 1 / (1 + np.exp(-params['beta'] * (n_high_p + params['risk_m']))) - government_effect_w
            if Pv > np.random.uniform(0.5, 1):
                self.risk = 'R'
            else:
                self.risk = 'NR'

    def update_o_people_gv(self, params, government_effect_o):
        """
        Updates the sentiment of 'o_people' type nodes by considering both the sentiment influence of their influencers and government effects.
        
        Args:
            params (dict): Includes parameters for the sentiment update calculation.
            government_effect_o (float): Government intervention effect that influences sentiment adjustment.
        """
        if self.type == 'o_people':
            r_1 = sum(1 if inf.risk == 'R' else -1 for inf in self.influencers) / (len(self.influencers) + 1)
            g_1 = 1 / (1 + np.exp(-params['theta'] * r_1)) if r_1 != 0 else 0
            g_1 = g_1 - government_effect_o  # Apply government effect
            if g_1 > np.random.uniform(0, 1):
                self.sentiment = np.random.choice(['L', 'H'])
            else:
                self.sentiment = 'M'
            
            sentiments = [inf.sentiment for inf in self.influencers if inf.sentiment]
            eta = sum(1 for inf in self.influencers if inf.sentiment == self.sentiment) / len(sentiments) if sentiments else 0
            if eta > self.sigma:
                sentiment_counter = Counter([inf.sentiment for inf in self.influencers if inf.sentiment])
                self.sentiment = max(sentiment_counter, key=sentiment_counter.get)

    def forward(self, params):
        """
        Processes one step of updates for this node, calling appropriate methods based on node type.
        
        Args:
            params (dict): Parameters required for updating node statuses.
        """
        self.update_m_media(params)
        self.update_w_media(params)
        self.update_o_people(params)

    def forward_gv(self, params, government_effect_w, government_effect_o):
        """
        Processes one step of updates for this node under government intervention, calling updates with government effects.
        
        Args:
            params (dict): Parameters required for updating node statuses.
            government_effect_w (float): Government effect applied to 'w_media' nodes.
            government_effect_o (float): Government effect applied to 'o_people' nodes.
        """
        self.update_m_media(params)
        self.update_w_media_gv(params, government_effect_w)
        self.update_o_people_gv(params, government_effect_o)



class Network:
    def __init__(self):
        """Initialize the network with an empty dictionary to store nodes."""
        self.nodes = {}

    def add_node(self, node):
        """Add a node to the network using the node's name as the key."""
        self.nodes[node.name] = node

    def calculate_sentiment_distribution(self):
        """Calculate the distribution of sentiments among 'o_people' type nodes."""
        sentiment_counts = Counter(node.sentiment for node in self.nodes.values() if node.type == 'o_people')
        return sentiment_counts

    def simulate_step(self, alpha, beta, theta):
        """Simulate a single step of the network dynamics with given parameters alpha, beta, and theta."""
        sentiment_counts = self.calculate_sentiment_distribution()
        property = self.calculate_state_proportions()
        risk_m = property['m_media']['R']
        d = (sentiment_counts['H'] + sentiment_counts['L'] - sentiment_counts['M']) / (sentiment_counts['M'] + 1)
        params = {
            'alpha': alpha, 'beta': beta, 'theta': theta, 'd': d,
            'n_high': sentiment_counts['H'], 'n_low': sentiment_counts['L'], 
            'n_middle': sentiment_counts['M'], 'risk_m': risk_m
        }
        for node in self.nodes.values():
            node.forward(params)
    
    def simulate_step_gv(self, alpha, beta, theta, government_effect_w, government_effect_o):
        """Simulate a single step considering government effects on 'w_media' and 'o_people'."""
        sentiment_counts = self.calculate_sentiment_distribution()
        property = self.calculate_state_proportions()
        risk_m = property['m_media']['R']
        d = (sentiment_counts['H'] + sentiment_counts['L'] - sentiment_counts['M']) / (sentiment_counts['M'] + 1)
        params = {
            'alpha': alpha, 'beta': beta, 'theta': theta, 'd': d,
            'n_high': sentiment_counts['H'], 'n_low': sentiment_counts['L'], 
            'n_middle': sentiment_counts['M'], 'risk_m': risk_m
        }
        for node in self.nodes.values():
            node.forward_gv(params, government_effect_w, government_effect_o)

    def simulate_steps(self, steps, alpha, beta, theta, delta_1, delta_2, delta_3, delta_4, gamma_1, gamma_2, gamma_3, gamma_4, cutoff_1=31, cutoff_2=51, duration_1=15, duration_2=15, duration_3=30):
        """Simulate multiple steps of the network dynamics over a specified number of steps and time intervals with varying government effects."""
        # Calculate initial distribution and setup initial history tracking
        middle_1 = round(cutoff_1 + 0.5 * duration_1)
        middle_2 = round(cutoff_2 + 0.5 * duration_2)
        middle_3 = round(cutoff_2 + 0.5 * (duration_3 + duration_2))
        ini_distribution = self.calculate_state_proportions()
        history = [ini_distribution]

        for t in range(steps):
            # Check for first special effect period
            if cutoff_1 <= t < cutoff_1 + duration_1 + 1:
                government_effect_w = delta_1 * ((t - middle_1) / duration_1) ** 2 + delta_2
                government_effect_o = gamma_1 * ((t - middle_1) / duration_1) ** 2 + gamma_2
                self.simulate_step_gv(alpha, beta, theta, government_effect_w, government_effect_o)
                proportions = self.calculate_state_proportions()
                history.append(proportions)

            # Check for second special effect period
            elif cutoff_2 <= t < cutoff_2 + duration_2 + duration_3 + 1:
                if t < cutoff_2 + duration_2 + 1:
                    government_effect_w = delta_3 * ((t - middle_2) / duration_2) ** 2 + delta_4
                    government_effect_o = gamma_3 * ((t - middle_3) / (duration_2 + duration_3)) ** 2 + gamma_4
                    self.simulate_step_gv(alpha, beta, theta, government_effect_w, government_effect_o)
                else:
                    government_effect_o = gamma_3 * ((t - middle_3) / (duration_2 + duration_3)) ** 2 + gamma_4
                    self.simulate_step_gv(alpha, beta, theta, 0, government_effect_o)

                proportions = self.calculate_state_proportions()
                history.append(proportions)

            else:
                # Regular simulation step
                self.simulate_step(alpha, beta, theta)
                proportions = self.calculate_state_proportions()
                history.append(proportions)

        return history

    def calculate_state_proportions(self):
        """Calculate the proportion of each state within each node type ('m_media', 'w_media', 'o_people')."""
        states = {
            'm_media': Counter({'R': 0, 'NR': 0}),
            'w_media': Counter({'R': 0, 'NR': 0}),
            'o_people': Counter({'H': 0, 'M': 0, 'L': 0})
        }

        for node in self.nodes.values():
            if node.type in ['m_media', 'w_media']:
                states[node.type][node.risk] += 1
            elif node.type == 'o_people':
                states[node.type][node.sentiment] += 1

        proportions = {type: {state: count / sum(states[type].values()) for state, count in states[type].items()} for type in states}
        return proportions

    def display(self):
        """Display the current status of each node in the network."""
        for name, node in self.nodes.items():
            print(f"{name} ({node.type}): Risk={node.risk}, Sentiment={node.sentiment}, Influencers={[inf.name for inf in node.influencers]}")



In [ ]:
import numpy as np
from collections import defaultdict

np.random.seed(1234)  # Set the random seed for reproducibility
network = Network()  # Create an instance of the Network class

# Probabilities for different node types
p_w_media = [345/348, 3/348]  # Probability distribution for w_media nodes
p_m_media = [1, 0]  # Probability distribution for m_media nodes (always 'R')
p_o_people = [169/537, 210/537, 158/537]  # Probability distribution for o_people nodes

# Create and add nodes from links data for origins
for origin_name, group in links.groupby(['origin_name']):
    node_type = group['o_type'].values[0]
    origin_name_ = group['origin_name'].values[0]

    if node_type == 'w_media':
        risk = risk_wemedia.get(origin_name_, None)
        if risk is None:  # Assign random risk if not present
            risk = np.random.choice(['R', 'NR'], p=p_w_media)
        node = Node(name=origin_name_, node_type=node_type, risk=risk)
        
    elif node_type == 'm_media':
        risk = risk_mainstream.get(origin_name_, None)
        if risk is None:
            risk = np.random.choice(['R', 'NR'], p=p_m_media)
        node = Node(name=origin_name_, node_type=node_type, risk='R')
        
    elif node_type == 'o_people':
        sentiment = np.random.choice(['H', 'M', 'L'], p=p_o_people)
        node = Node(name=origin_name_, node_type=node_type, sentiment=sentiment, sigma=np.random.random())

    network.add_node(node)

# Create and add nodes from links data for users
for user_name, group in links.groupby(['user_name']):
    node_type = group['d_type'].values[0]
    user_name_ = group['user_name'].values[0]

    if node_type == 'o_people':
        sentiment = np.random.choice(['H', 'M', 'L'], p=p_o_people)
        node = Node(name=user_name_, node_type=node_type, sentiment=sentiment, sigma=np.random.random())
        
    elif node_type == 'm_media':
        risk = risk_mainstream.get(user_name_, None)
        if risk is None:
            risk = np.random.choice(['R', 'NR'], p=p_m_media)
        node = Node(name=user_name_, node_type=node_type, risk=risk)
        
    else:  # Assume w_media or another unspecified category
        risk = risk_wemedia.get(user_name_, None)
        if risk is None:
            risk = np.random.choice(['R', 'NR'], p=p_w_media)
        node = Node(name=user_name_, node_type=node_type, risk=risk)
        
    network.add_node(node)

# Set up bidirectional influencer relationships
for origin_name, user_name in links[['origin_name', 'user_name']].values:
    object_1 = network.nodes[origin_name]
    object_2 = network.nodes[user_name]
    object_1.add_influencer(object_2)
    object_2.add_influencer(object_1)

In [ ]:
joblib.dump(network, 'network_sigmoid_wt_v4.pkl')

In [ ]:
# best_param = results_df[results_df['error'] == results_df['error'].min()]['param']
# network_ini = joblib.load('network.pkl')
# history = network_ini.simulate_steps(95, *best_param.values[0])

In [ ]:
# 保存nodes为json文件
# with open('nodes.json', 'w') as f:
#     json.dump(nodes, f, ensure_ascii=False, indent=4)

## Model Calibration

### Step 1: Setup and Model Definition

The model setup and definitions, including classes for nodes and networks, have been provided. We first need to ensure that all necessary libraries are imported at the beginning of the script. I noticed that `numpy` is used as `np` in the code, but it was not explicitly imported, so we have added this import.

### Step 2: Parameter Sampling

Define a function to sample parameter combinations uniformly. Sampling should be done in the promising regions of the parameter space for `alpha`, `beta`, and `theta`. Since the specific promising regions are not provided, I have used example ranges for demonstration, but these should be adjusted according to your prior analysis or the specific content of Supplementary Material Figure 13.

### Step 3: Simulation Execution

For each parameter combination, we will use a set of predetermined random seeds to run the simulation and compare the simulation results with the empirical dataset (`m_media_risk_rate`, `w_media_risk_rate`, `o_people_middle_rate`).

### Step 4: Selection of Parameter Combinations

After running the simulations, we will apply a specified error threshold to select the acceptable parameter combinations.



In [ ]:

def sample_parameter_combinations_lhs(num_samples=10000):
    """
    Generate parameter combinations using Latin Hypercube Sampling (LHS).
    
    Parameters:
        num_samples (int): The number of parameter samples to generate.
        
    Returns:
        list: A list of tuples, each containing a sampled parameter combination.
    """
    sampler = LatinHypercube(d=11)  # Dimensionality of parameter space
    samples = sampler.random(n=num_samples)

    # Mapping samples from [0, 1] interval to the actual parameter ranges
    alpha_range = (2, 5)
    beta_range = (10, 20)
    theta_range = (0.01, 0.4)
    delta_ranges = [(-1, -0.01), (0.1, 0.2)] * 2
    gamma_ranges = [(-1, -0.01), (0.1, 0.2)] * 2

    # Unpack all parameters and map them to their respective ranges
    parameters = [alpha_range, beta_range, theta_range] + delta_ranges + gamma_ranges
    return [
        tuple(sample[i] * (param_range[1] - param_range[0]) + param_range[0] for i, param_range in enumerate(parameters))
        for sample in samples
    ]

def simulate_network_for_parameters(network, param, num_seeds=3):
    """
    Run network simulation for given parameters across multiple random seeds and summarize the results.
    
    Parameters:
        network: The network instance to simulate.
        param (tuple): The parameters for the simulation.
        num_seeds (int): Number of different seeds to simulate.
        
    Returns:
        dict: A dictionary containing the fit status, parameters used, and the average error.
    """
    errors = []
    for seed in range(num_seeds):
        np.random.seed(seed)
        # Reset network to initial state if necessary
        simulation_results = network.simulate_steps(95, *param)  # Assuming simulation for 96 days

        # Process simulation results weekly
        weekly_results = {i: {} for i in range(14)}  # Preparing dictionary to store weekly data
        for i in range(14):
            week_slice = simulation_results[i*7:(i+1)*7]
            weekly_results[i] = {
                'm_media_risk': np.mean([day['m_media']['R'] for day in week_slice]),
                'w_media_risk': np.mean([day['w_media']['R'] for day in week_slice]),
                'o_people_high': np.mean([day['o_people']['H'] for day in week_slice]),
                'o_people_middle': np.mean([day['o_people']['M'] for day in week_slice])
            }

        # Calculate error comparing to empirical data
        error = calculate_error(weekly_results, empirical_data)
        errors.append(error)

    avg_error = np.mean(errors)
    return {
        'result': 'fitting' if meets_error_criteria(avg_error) else 'not fitting',
        'param': param,
        'error': avg_error
    }

def calculate_error(simulation_results, empirical_data):
    """
    Calculate error between simulation results and empirical data.
    
    Parameters:
        simulation_results (dict): The simulated data.
        empirical_data (dict): The observed empirical data.
        
    Returns:
        float: The average absolute error.
    """
    error_sum = sum(
        abs(simulation_results[key][step] - empirical_data[key][step])
        for key in simulation_results
        for step in simulation_results[key]
    )
    return error_sum / 48  # Total number of time steps considered

def meets_error_criteria(error):
    """
    Check if the given error meets the acceptable threshold.
    
    Parameters:
        error (float): The calculated error.
        
    Returns:
        bool: True if error is within acceptable limits, else False.
    """
    error_threshold = 0.07  # Example threshold, adjust based on requirements
    return error < error_threshold

def plot_history_with_empirical(history, empirical_data):
    """
    Plot the simulation history alongside empirical data points.
    
    Parameters:
        history (list): List of dictionaries containing the simulated state proportions over time.
        empirical_data (dict): Dictionary containing empirical data points.
    """
    plt.figure(figsize=(10, 6))
    labels = {'m_media_R': 'Mainstream Media Risk', 'w_media_R': 'Web Media Risk', 'o_people_H': 'High Sentiment', 'o_people_M': 'Middle Sentiment', 'l_people_L': 'Low Sentiment'}
    time_steps = range(len(history))

    # Plotting simulated data
    for label, data_key in labels.items():
        plt.plot(time_steps, [day.get(data_key, 0) for day in history], label=label)

    # Plot empirical data points
    for week, data_point in empirical_data.items():
        day = week * 7
        plt.scatter([day] * len(data_point), list(data_point.values()), label=f'Week {week} Empirical Data')

    plt.legend()
    plt.title('Comparison of Simulated and Empirical Data')
    plt.xlabel('Time (days)')
    plt.ylabel('Proportion')
    plt.grid(True)
    plt.show()


In [ ]:

# Load empirical data from CSV files
w_media_empirical = pd.read_csv('./output/empirical/w_media_risk_data_7days.csv')
o_people_empirical = pd.read_csv('./output/empirical/sentiment_data_7days.csv')
m_media_empirical = pd.read_csv('./output/empirical/m_media_risk_data_7days.csv')

# Convert data into dictionaries mapping period identifiers to risk or sentiment proportions
emp_w_risk_p = dict(zip(w_media_empirical['period_id_3d'], w_media_empirical['risk_p']))
emp_m_risk_p = dict(zip(m_media_empirical['period_id_3d'], m_media_empirical['risk_p']))
emp_sentiment_high_p = dict(zip(o_people_empirical['period_id_3d'], o_people_empirical['high_p']))
emp_sentiment_middle_p = dict(zip(o_people_empirical['period_id_3d'], o_people_empirical['middle_p']))
emp_sentiment_low_p = dict(zip(o_people_empirical['period_id_3d'], o_people_empirical['low_p']))

# Aggregate the empirical data into a single dictionary for ease of use in simulations and analysis
empirical_data = {
    'w_risk_p': emp_w_risk_p,
    'm_risk_p': emp_m_risk_p,
    'sentiment_high_p': emp_sentiment_high_p,
    'sentiment_middle_p': emp_sentiment_middle_p,
    # Uncomment the next line if low sentiment proportions are needed for further analysis
    # 'sentiment_low_p': emp_sentiment_low_p
}


In [ ]:
results = []

In [ ]:
np.random.seed(2345)
all_params = sample_parameter_combinations_lhs(5000)

In [ ]:
# Load pre-initialized network from a saved file outside of the loop to avoid repetitive loading if applicable
network = joblib.load('network_sigmoid_wt_v4.pkl')

# Initialize an empty list to store simulation results
results = []

# Loop through all parameters starting from the 5508th parameter using tqdm for progress tracking
for param in tqdm.tqdm(all_params[5507:], desc='Simulating'):
    try:
        # Perform network simulation with the current parameter set
        result = simulate_network_for_parameters(network, param, num_seeds=3)
        
        # Store the result of the simulation
        results.append(result)
        
        # Print the parameters and results if the simulation was successful
        if result['result'] != 'not fitting':
            print(f"Successful Parameter Set: {param}, Result: {result}")
    
    except Exception as e:
        # Print error message if an exception occurs
        print(f"Error occurred for parameter {param}: {e}")

In [ ]:
results_df = pd.DataFrame(results).sort_values('error')
results_df['alpha'] = results_df['param'].apply(lambda x: x[0])
results_df['beta'] = results_df['param'].apply(lambda x: x[1])
results_df['theta'] = results_df['param'].apply(lambda x: x[2])
results_df['delta_1'] = results_df['param'].apply(lambda x: x[3])
results_df['delta_2'] = results_df['param'].apply(lambda x: x[4])
results_df['delta_3'] = results_df['param'].apply(lambda x: x[5])
results_df['delta_4'] = results_df['param'].apply(lambda x: x[6])
results_df['gamma_1'] = results_df['param'].apply(lambda x: x[7])
results_df['gamma_2'] = results_df['param'].apply(lambda x: x[8])
results_df['gamma_3'] = results_df['param'].apply(lambda x: x[9])
results_df['gamma_4'] = results_df['param'].apply(lambda x: x[10])
results_df['error'].min()

In [ ]:
results_df.to_csv('./results_df.csv', index=False)

In [ ]:
results_df = pd.read_csv('./results_df.csv')

In [ ]:
results_df[results_df['error'] == results_df['error'].min()]['param'].values[0]

# 将字符串转换为元组
import ast
best_param = ast.literal_eval(results_df[results_df['error'] == results_df['error'].min()]['param'].values[0])
alpha, beta, theta, delta_1, delta_2, delta_3, delta_4, gamma_1, gamma_2, gamma_3, gamma_4 = best_param

In [ ]:
empirical_data_another = {
    'w_risk_p': emp_w_risk_p,
    'm_risk_p': emp_m_risk_p,
    'sentiment_high_p': emp_sentiment_high_p,
    'sentiment_middle_p': emp_sentiment_middle_p,
    'sentiment_low_p': emp_sentiment_low_p
}

In [ ]:
# alpha, beta, theta = (3.17716490514395, 14.9066214771117403, 10.10791529106301318)
# param = alpha, beta, theta
# print(np.random.uniform(0.1, 0.15))
network_ini = joblib.load('network_sigmoid_wt_v4.pkl')

history = network_ini.simulate_steps(91, alpha, beta, theta, delta_1, delta_2, delta_3, delta_4, gamma_1, gamma_2, gamma_3, gamma_4)

plot_history_with_empirical(history, empirical_data_another)

In [ ]:
history

# 我要计算每个情绪状态的误差情况
error_for_sentiment_high = 0
error_for_sentiment_middle = 0
error_for_sentiment_low = 0

for i in range(14):
    error_for_sentiment_high += abs(emp_sentiment_high_p[i] - history[i]['o_people']['H'])
    error_for_sentiment_middle += abs(emp_sentiment_middle_p[i] - history[i]['o_people']['M'])
    error_for_sentiment_low += abs(emp_sentiment_low_p[i] - history[i]['o_people']['L'])

In [ ]:
# 计算每个媒体风险的误差情况
error_for_w_risk = 0
error_for_m_risk = 0

#注意有些时间步没有数据，所以要判断一下
for i in range(14):
    if i in emp_w_risk_p:
        error_for_w_risk += abs(emp_w_risk_p[i] - history[i]['w_media']['R'])
    if i in emp_m_risk_p:
        error_for_m_risk += abs(emp_m_risk_p[i] - history[i]['m_media']['R'])


In [ ]:
# 计算各自的平均误差，注意有些长度不是14，并打印
print(f'error_for_sentiment_high: {error_for_sentiment_high/14}')
print(f'error_for_sentiment_middle: {error_for_sentiment_middle/14}')
print(f'error_for_sentiment_low: {error_for_sentiment_low/14}')
print(f'error_for_w_risk: {error_for_w_risk/14}')
print(f'error_for_m_risk: {error_for_m_risk/6}')

In [ ]:
empirical_data_another = {
    'w_risk_p': emp_w_risk_p,
    'm_risk_p': emp_m_risk_p,
    'sentiment_high_p': emp_sentiment_high_p,
    'sentiment_middle_p': emp_sentiment_middle_p,
    'sentiment_low_p': emp_sentiment_low_p
}

In [ ]:
# 找到最小error的参数
results_df = pd.DataFrame(results)
results_df.to_csv('results_df.csv', index=False)